# Compute RBS homology

In [ ]:
import oat_python as oat

import copy
import plotly.graph_objects as go
import numpy as np
import networkx as nx
import hypernetx as hnx
import sklearn

# Define a hypergraph

In [ ]:
# define a hypergraph

E = { "A": ["x"], "B": ["y"], "C": ["x","y","z",], "D": ["x","y","w",], }

# Plot

### Initial hypergraph

In [ ]:
hnxgraph = hnx.Hypergraph(E)
hnx.drawing.draw(hnxgraph)

# Compute homology

In [ ]:
# convert to a list of edges
G                           =   list( E.values() )
# graph representing the edge containment poset (forgetting direction)
containment                 =   oat.hypergraph.edge_containment_graph_symmetrized( G )
# graph whose edges form the set complement of the containment graph
anticontainment             =   nx.complement( containment )
# adjacency matrix of the anticontainment graph
anti_adjacency              =   nx.adjacency_matrix( anticontainment ).todense()
dissimilarity_matrtix       =   oat.dissimilarity.matrix_from_dense(
                                    dissimilarity_matrix    =   anti_adjacency,
                                    dissimilarity_max       =   0.5
                                )

# factored boundary matrix
factored                    =   oat.rust.FactoredBoundaryMatrixVr(
                                    dissimilarity_matrix    =   dissimilarity_matrtix,
                                    homology_dimension_max  =   1, 
                                )

In [ ]:
homology        =   factored.homology(
                        return_cycle_representatives    =   True,
                        return_bounding_chains          =   True,
                    )
homology

# Inspect a cycle representative

In [ ]:
cycle           =   homology["cycle representative"][1]
cycle

### Relabel each vertex with an edge

Recall that each vertex in RBS homology represents an edge in the reduced hypergraph (which corresponds to a set of hyperedges in the initial hypergraph).  Here we relabel each vertex with **one** of the edges that maps to it.

In [ ]:

remap = { p: k for p,k in enumerate(E.keys()) }

cycle               =   copy.deepcopy(homology["cycle representative"][1])
cycle["simplex"]    =   [ [remap[x] for x in simplex] for simplex in cycle["simplex"] ]

cycle

# Plot a cycle representative

In [ ]:
edges   =   homology["cycle representative"][1]["simplex"] # the edges in the cycle
coo     =   oat.plot.hop_mds_from_simplices( edges.tolist() ) # coordinates generated by MDS

data    =   []
for edge in edges:
    trace   =   oat.plot.edge__trace3d( edge=edge, coo=coo )
    label   =   "Simplex " + str( [ remap[x] for x in edge] )
    trace.update( name=label, text=label )
    data.append(trace)

fig = go.Figure(data)
fig.show()



# Suspend and repeat

In [ ]:
F = { "A": ["x"], "B": ["y"], "C": ["x","y","z",], "D": ["x","y","w",], "E":["x","y","z","w","a"], "F":["x","y","z","w","b"] }
hnxgraph = hnx.Hypergraph(F)
hnx.drawing.draw(hnxgraph)

In [ ]:
# convert to a list of edges
G                           =   list( F.values() )
# graph representing the edge containment poset (forgetting direction)
containment                 =   oat.hypergraph.edge_containment_graph_symmetrized( G )
# graph whose edges form the set complement of the containment graph
anticontainment             =   nx.complement( containment )
# adjacency matrix of the anticontainment graph
anti_adjacency              =   nx.adjacency_matrix( anticontainment ).todense()
dissimilarity_matrtix       =   oat.dissimilarity.matrix_from_dense(
                                    dissimilarity_matrix    =   anti_adjacency,
                                    dissimilarity_max       =   0.5
                                )

# factored boundary matrix
factored                    =   oat.rust.FactoredBoundaryMatrixVr(
                                    dissimilarity_matrix    =   dissimilarity_matrtix,
                                    homology_dimension_max  =   2, 
                                )

# print homology
homology        =   factored.homology(
                        return_cycle_representatives    =   True,
                        return_bounding_chains          =   True,
                    )
display(homology)

# print a cycle
cycle                       =   homology["cycle representative"][1]
display(cycle)

# print cycle with reverted vertex labels
remap                       =   { p: k for p,k in enumerate(F.keys()) }
cycle                       =   copy.deepcopy(homology["cycle representative"][1])
cycle["simplex"]            =   [ [remap[x] for x in simplex] for simplex in cycle["simplex"] ]
display(cycle)


In [ ]:
triangles   =   homology["cycle representative"][1]["simplex"] # the edges in the cycle
coo         =   oat.plot.hop_mds_from_simplices( triangles.tolist() ) # coordinates generated by MDS

data        =   []
for triangle in triangles:
    trace   =   oat.plot.triangle__trace3d( triangle=triangle, coo=coo )
    label   =   "Simplex " + str( [ remap[x] for x in edge] )
    trace.update( name=label, text=label, opacity=1.0 )
    data.append(trace)

fig = go.Figure(data)
fig.update_layout(height=900, width=900,)
fig.show()



In [ ]:
triangles   =   homology["cycle representative"][1]["simplex"] # the edges in the cycle
coo         =   oat.plot.hop_mds_from_simplices( triangles.tolist() ) # coordinates generated by MDS

data        =   []
for triangle in triangles:
    trace   =   oat.plot.triangle__trace3d( triangle=triangle, coo=coo )
    label   =   "Simplex " + str( [ remap[x] for x in edge] )
    trace.update( name=label, text=label, opacity=1.0, color="grey", )
    data.append(trace)

edges = list(oat.simplex.dnfaces( triangles, facedim=1))
for edge in edges:
    trace   =   oat.plot.edge__trace3d( edge, coo=coo )
    trace.update( showlegend=False, hoverinfo='none', line=dict(width=5, color="white"))
    data.append(trace)    

fig = go.Figure(data)

fig.update_layout( title=dict(text="Octohedron"), width=900, height=800, template="plotly_dark",)
fig.update_layout(scene = dict(     xaxis = dict(showgrid = False,showticklabels = False, ),
                                    yaxis = dict(showgrid = False,showticklabels = False),
                                    zaxis = dict(showgrid = False,showticklabels = False)
             ))

fig.show()

